In [1]:
import sys
import os
import numpy as np
import cv2
import openslide
from openslide import open_slide
from openslide.deepzoom import DeepZoomGenerator
import xml.etree.ElementTree as ET
from xml.dom import minidom
import geojson
import argparse
import matplotlib.pyplot as plt
import fastai
from fastai.vision.all import *
import PIL
matplotlib.use('Agg')
import pandas as pd
import datetime
from skimage import draw, measure, morphology, filters
from shapely.geometry import Polygon, Point, MultiPoint, MultiPolygon, shape
from shapely.ops import cascaded_union, unary_union
import json
import shapely
import warnings
from scipy import ndimage
sys.path.insert(0, '../Utils/')
from Utils import generate_deepzoom_tiles, extract_tile_start_end_coords
from Utils import do_mask_original,check_tissue,whitespace_check
from Utils import slide_ROIS
from Utils import get_downsample_factor, get_image_at_target_mag
from Utils import create_dir_if_not_exists
from Utils import get_map_startend
from Utils import cancer_mask_fix_res, tile_ROIS, check_any_invalid_poly, make_valid_poly
from Utils import convert_img
warnings.filterwarnings("ignore")

In [2]:
############################################################################################################
#USER INPUT 
############################################################################################################
mag_extract = 20        # do not change this, model trained at 250x250 at 20x
save_image_size = 250   # do not change this, model trained at 250x250 at 20x
pixel_overlap = 100       # specify the level of pixel overlap in your saved images
limit_bounds = True     # this is weird, dont change it
smooth = True           # whether or not to gaussian smooth the output probability map
ft_model = True        # whether or not to use fine-tuned model
mag_target_prob = 2.5   # 2.5x for probality maps
mag_target_tiss = 1.25   #1.25x for tissue detection
bi_thres = 0.4  #Binary classification threshold for cancer mask

############################################################################################################
#DIR
############################################################################################################
proj_dir = '/fh/fast/etzioni_r/Lucas/mh_proj/mutation_pred/'
wsi_location_ccola = proj_dir + '/data/CCola/all_slides/'
wsi_location_opx = proj_dir + '/data/OPX/'
out_location = proj_dir + 'intermediate_data/cancer_prediction_results110224/'+ "IMSIZE" + str(save_image_size) + "_OL" + str(pixel_overlap) + "/"
model_path = proj_dir + 'models/cancer_detection_models/mets/'

In [3]:
############################################################################################################
#Select IDS
############################################################################################################
#Get IDs that are in FT train or already processed to exclude 
fine_tune_ids_df = pd.read_csv(proj_dir + '/intermediate_data/cd_finetune/cancer_detection_training/all_tumor_fraction_info.csv')
ft_train_ids = list(fine_tune_ids_df.loc[fine_tune_ids_df['Train_OR_Test'] == 'Train','sample_id'])

#OPX_182 –Exclude Possible Colon AdenoCa 
toexclude_ids = ft_train_ids + ['OPX_182'] #25

#All available IDs
opx_ids = [x.replace('.tif','') for x in os.listdir(wsi_location_opx)] #207
opx_ids.sort()
ccola_ids = [x.replace('.svs','') for x in os.listdir(wsi_location_ccola) if '(2017-0133)' in x] #234
ccola_ids.sort()
#all_test = opx_ids + ccola_ids
all_test = opx_ids

#Exclude ids in ft_train or processed
selected_ids = [x for x in all_test if x not in toexclude_ids] #416

In [20]:
cur_id = selected_ids[0]
if 'OPX' in cur_id:
    _file = wsi_location_opx + cur_id + ".tif"
    rad_tissue = 5
elif '(2017-0133)' in cur_id:
    _file = wsi_location_ccola + cur_id + '.svs'
    rad_tissue = 2


#Load slides
oslide = openslide.OpenSlide(_file)
save_name = str(Path(os.path.basename(_file)).with_suffix(''))

if ft_model == True:
    learn = load_learner(model_path + 'ft_models/dlv3_2ep_2e4_update-07182023_RT_fine_tuned..pkl',cpu=False) #all use mets model
else:
    learn = load_learner(model_path + 'dlv3_2ep_2e4_update-07182023_RT.pkl',cpu=False) #all use prior mets model

#Load tile info 
tile_info_df = pd.read_csv(out_location + cur_id + "/"  + save_name + "_tiles.csv")
tile_mag_extract = list(set(tile_info_df['MAG_EXTRACT']))[0]
tile_save_image_size = list(set(tile_info_df['SAVE_IMAGE_SIZE']))[0]
tile_pixel_overlap = list(set(tile_info_df['PIXEL_OVERLAP']))[0]
tile_limit_bounds =   list(set(tile_info_df['LIMIT_BOUNDS']))[0]



#Generate tiles
tiles, tile_lvls, physSize, base_mag = generate_deepzoom_tiles(oslide,save_image_size, pixel_overlap, limit_bounds)

print('starting inference')
#get level 0 size in px
l0_w = oslide.level_dimensions[0][0]
l0_h = oslide.level_dimensions[0][1]



#2.5x for probability maps
lvl_resize = get_downsample_factor(base_mag,target_magnification = mag_target_prob) #downsample factor
x_map = np.zeros((int(np.ceil(l0_h/lvl_resize)),int(np.ceil(l0_w/lvl_resize))), float)
x_count = np.zeros((int(np.ceil(l0_h/lvl_resize)),int(np.ceil(l0_w/lvl_resize))), float)

tile_info_df['pred_map_location'] = pd.NA

starting inference


In [23]:
print("WSI lvl0: ",(l0_h,l0_w))
print("x_map:",x_map.shape)
print("tile info: ",tile_info_df.shape)

WSI lvl0:  (19456, 24576)
x_map: (1216, 1536)
tile info:  (1091, 10)


In [40]:
outputs_np[1]

array([[1.5457438e-03, 4.4327467e-03, 1.2643543e-02, ..., 3.5962537e-01,
        4.1074625e-01, 4.6386924e-01],
       [5.4253079e-04, 1.9019608e-03, 6.6450899e-03, ..., 3.5119483e-01,
        3.8368952e-01, 4.1725668e-01],
       [1.9029516e-04, 8.1489264e-04, 3.4824344e-03, ..., 3.4285611e-01,
        3.5733429e-01, 3.7207744e-01],
       ...,
       [6.1148491e-08, 8.7884172e-07, 1.2630774e-05, ..., 3.3336130e-03,
        4.3537235e-03, 5.6841900e-03],
       [1.9062824e-07, 1.9137631e-06, 1.9212430e-05, ..., 4.0506353e-03,
        5.3119785e-03, 6.9633173e-03],
       [5.9426645e-07, 4.1673561e-06, 2.9223398e-05, ..., 4.9211048e-03,
        6.4797541e-03, 8.5278004e-03]], dtype=float32)

In [26]:
for index, row in tile_info_df.iterrows():
    if (index % 500 == 0): print(index)
    cur_xy = row['TILE_XY_INDEXES'].strip("()").split(", ")
    x ,y = int(cur_xy[0]) , int(cur_xy[1])
    
    #Extract tile for prediction
    lvl_in_deepzoom = tile_lvls.index(mag_extract)
    tile_pull = tiles.get_tile(lvl_in_deepzoom, (x, y))
    tile_pull = tile_pull.resize(size=(save_image_size, save_image_size),resample=PIL.Image.LANCZOS) #resize
    tile_starts, tile_ends, save_coords, tile_coords = extract_tile_start_end_coords(tiles, lvl_in_deepzoom, x, y) #get tile coords
    map_xstart, map_xend, map_ystart, map_yend = get_map_startend(tile_starts,tile_ends,lvl_resize) #Get current tile position in map
    tile_info_df.loc[index,'pred_map_location'] = str(tuple([map_xstart, map_xend, map_ystart, map_yend]))
    
    #Cancer segmentation
    tile_pull = np.array(tile_pull)
    with learn.no_bar():
        inp, pred_class, pred_idx, outputs = learn.predict(tile_pull[:, :, 0:3], with_input=True)
    
    #Get predicted output
    #NOTe: updated 11/06, use cv2.resize
    outputs_np = outputs.numpy() #[N_CLASS, IMAGE_SIZE, IMAGE_SIZE]
    output_c1_np = cv2.resize(outputs_np[1], (map_yend - map_ystart,map_xend - map_xstart)) #class1 predicted prob, resize (width (col in np), height(row in np))
    output_c1_np = output_c1_np.round(2)
    
    #Store predicted probabily in map and count
    try: 
        x_count[map_xstart:map_xend,map_ystart:map_yend] += 1
        x_map[map_xstart:map_xend,map_ystart:map_yend] += output_c1_np
    except:
        pass

0
500
1000


In [31]:
print('post-processing')
print('Cancer Prob generation')
x_count = np.where(x_count < 1, 1, x_count)
x_map = x_map / x_count
x_map[x_map>1]=1

if smooth == True:
    x_sm = filters.gaussian(x_map, sigma=2)
if smooth == False:
    x_sm = x_map
cmap = plt.get_cmap('jet')
rgba_img = cmap(x_sm)
rgb_img = np.delete(rgba_img, 3, 2)
colimg = PIL.Image.fromarray(np.uint8(rgb_img * 255))
colimg.shape

post-processing
Cancer Prob generation


(1216, 1536)

In [32]:
print('Get cancer mask')
print('detecting tissue')
#1.25x tissue detection
lvl_resize_tissue = get_downsample_factor(base_mag,target_magnification = mag_target_tiss) #downsample factor
lvl_img = get_image_at_target_mag(oslide,l0_w, l0_h,lvl_resize_tissue)
tissue, he_mask = do_mask_original(lvl_img, lvl_resize_tissue, rad = rad_tissue)

#Binary classification
binary_preds = cancer_mask_fix_res(x_sm,cv2.resize(np.uint8(he_mask),(x_sm.shape[1],x_sm.shape[0])), bi_thres)

Get cancer mask
detecting tissue


In [41]:
binary_preds.shape

(1216, 1536)

In [34]:
#Output annotation
print('saving...')
polygons = tile_ROIS(mask_arr=binary_preds, lvl_resize=lvl_resize)

#Make valid polygons (ex: OPX_022)
invalid_polygons = check_any_invalid_poly(polygons) #check if there is any invalid polys
if len(invalid_polygons) > 0 :
    polygons = make_valid_poly(polygons, buff_value = 4)

saving...


In [36]:
tile_info_df

,SAMPLE_ID,MAG_EXTRACT,SAVE_IMAGE_SIZE,PIXEL_OVERLAP,LIMIT_BOUNDS,TILE_XY_INDEXES,TILE_COOR_ATLV0,WHITE_SPACE,TISSUE_COVERAGE,pred_map_location
0,OPX_001,20,250,100,True,"(54, 9)",16100-2600_500-500,0.412144,0.957248,"(163, 194, 1007, 1038)"
1,OPX_001,20,250,100,True,"(55, 9)",16400-2600_500-500,0.358944,0.997344,"(163, 194, 1025, 1057)"
2,OPX_001,20,250,100,True,"(51, 10)",15200-2900_500-500,0.444496,0.996064,"(182, 213, 950, 982)"
3,OPX_001,20,250,100,True,"(52, 10)",15500-2900_500-500,0.457760,1.000000,"(182, 213, 969, 1000)"
4,OPX_001,20,250,100,True,"(53, 10)",15800-2900_500-500,0.360992,1.000000,"(182, 213, 988, 1019)"
...,...,...,...,...,...,...,...,...,...,...
1086,OPX_001,20,250,100,True,"(18, 57)",5300-17000_500-500,0.534944,0.949344,"(1063, 1094, 332, 363)"
1087,OPX_001,20,250,100,True,"(9, 58)",2600-17300_500-500,0.300464,0.972864,"(1082, 1113, 163, 194)"
1088,OPX_001,20,250,100,True,"(10, 58)",2900-17300_500-500,0.340288,0.969152,"(1082, 1113, 182, 213)"
1089,OPX_001,20,250,100,True,"(11, 58)",3200-17300_500-500,0.254416,1.000000,"(1082, 1113, 200, 232)"


In [ ]:
#Get binary prediction for each tile
#NOTE: previous do x_map when prediction, is not accurate, because the x_map may change as process to the next tile, so need to do this in post-processing
tile_info_df['TUMOR_PIXEL_PERC'] = pd.NA
for index, row in tile_info_df.iterrows():
    cur_map_loc = row['pred_map_location'].strip("()").split(", ")
    map_xstart, map_xend, map_ystart, map_yend = int(cur_map_loc[0]),int(cur_map_loc[1]), int(cur_map_loc[2]), int(cur_map_loc[3])

    #Get current prediction
    cur_pred = binary_preds[map_xstart:map_xend,map_ystart:map_yend]
    cur_count1 = np.sum(cur_pred == 1) #num pixels that has predicted prob = 1
    cur_perc1  = (cur_count1 / cur_pred.size) #fraction of pixels prob = 1
    tile_info_df.loc[index,'TUMOR_PIXEL_PERC'] = cur_perc1

In [ ]:

    
        if can_proceed == True:
    
    
            #Get binary prediction for each tile
            #NOTE: previous do x_map when prediction, is not accurate, because the x_map may change as process to the next tile, so need to do this in post-processing
            tile_info_df['TUMOR_PIXEL_PERC'] = pd.NA
            for index, row in tile_info_df.iterrows():
                cur_map_loc = row['pred_map_location'].strip("()").split(", ")
                map_xstart, map_xend, map_ystart, map_yend = int(cur_map_loc[0]),int(cur_map_loc[1]), int(cur_map_loc[2]), int(cur_map_loc[3])
    
                #Get current prediction
                cur_pred = binary_preds[map_xstart:map_xend,map_ystart:map_yend]
                cur_count1 = np.sum(cur_pred == 1) #num pixels that has predicted prob = 1
                cur_perc1  = (cur_count1 / cur_pred.size) #fraction of pixels prob = 1
                tile_info_df.loc[index,'TUMOR_PIXEL_PERC'] = cur_perc1
    
            # tile_info_df.to_csv(save_location + save_name + "_TILE_TUMOR_PERC.csv", index = False)
    
    
            # #Grab tiles and plot
            # tile_info_df_sorted = tile_info_df.sort_values(by = ['TUMOR_PIXEL_PERC'], ascending = False) 
            # for i in range(0,5): #top5
            #     cur_row = tile_info_df_sorted.iloc[i]
            #     cur_xy = cur_row['TILE_XY_INDEXES'].strip("()").split(", ")
            #     x ,y = int(cur_xy[0]) , int(cur_xy[1])
            #     tile_pull_ex = tiles.get_tile(tile_lvls.index(mag_extract), (x, y))
            #     tile_pull_ex = tile_pull_ex.resize(size=(save_image_size, save_image_size),resample=PIL.Image.LANCZOS) #resize
            #     tile_pull_ex = convert_img(tile_pull_ex)
    
            #     #Save tile
            #     cur_tf = round(cur_row['TUMOR_PIXEL_PERC'],2)
            #     cur_mag = cur_row['MAG_EXTRACT']
            #     tile_save_name = "TILE_@" + str(cur_mag) + "x" + "_X" + str(x) +  "Y" + str(y) +   "_TF" + str(cur_tf) + ".png"
            #     tile_pull_ex.save(os.path.join(save_location, tile_save_name))